# Programming Drill 2.7.1 Write a function that accepts two matrices and constructs their tensor product.

Yanofsky, Noson S.. Quantum Computing for Computer Scientists (p. 73). Cambridge University Press. Kindle Edition. 

We will continue from where we left off. We will reuse the last class we created and then add the tensor product (Which is Kroneker's prodict for this case to it)

In [57]:
import numpy as np
from numba import jit # Not used yet. Coming!

In [58]:
class Vector(object):
    """
    Similar like the previous chapter examples, we define a Vector class now
    
    The method names and their work is self explainatory
    """
    def __init__(self, elements: list):
        self.vector = np.array(elements)
    
    def __add__(self, other):
        """
        Over-riding the '+' operation for vectors
        """
        return Vector(self.vector + other.vector)
    
    def __sub__(self, other):
        return Vector(self.vector - other.vector)
    
    def __str__(self):
        st = np.array2string(self.vector, separator=',')
        
        return "Vector "+st
    
    def __radd__(self, other):
        """
        And the reverse add operation as well.
        """
        return Vector(self.vector + other)
    
    def __mul__(self, other):
        """
        Over-riding the * operation
        """
        rows1, cols1 = self.vector.shape
        rows2, cols2 = other.vector.shape
        if cols1 != rows2:
            print ("The dimentions {} and {} can not be multiplied".format(self.vector.shape, other.vector.shape))
            return None
        return Vector(np.matmul(self.vector, other.vector))
    
    def __eq__(self, other):
        return (self.vector == other.vector).all()
    
    def scalar_multiply(self, scalar):
        return Vector(scalar * self.vector)
    
    def additive_inverse(self):
        return Vector(self.scalar_multiply(-1))
    
    def dim(self):
        return self.vector.shape
    
    def t(self):
        """
        Stealing from PyTorch :)
        """
        return Vector(self.vector.transpose())
    
    def dagger(self):
        return Vector(self.vector.transpose().conjugate())
    
    def inner(self, other):
        """
        By definition it returns a scalar.
        """
        inp = self.dagger() * other
        return inp.vector[0, 0]
    
    def norm(self):
        """
        Using the default Frobenius norm
        """
        return np.linalg.norm(self.vector)
    
    def dist(self, other):
        return (self - other).norm()
    
    def is_hermitian(self):
        """
        Using numpy `allclose` is a better option due to floating point artihmatic
        """
        daggered = self.dagger()
        return (self.vector == daggered.vector).all()
    
    def is_unitary(self):
        rows, cols = self.vector.shape
        if rows != cols:
            print("Can not check unitary on non square matrix")
            return False
        u_udag = self * self.dagger()
        udag_u = self.dagger() * self
        identity = np.identity(rows, dtype=np.complex128)
        return (np.allclose(u_udag.vector, identity) and np.allclose(udag_u.vector, identity))

    def tensor_product(self, other):
        return Vector(np.kron(self.vector, other.vector))

Let's do one exercise (2.7.7)

In [59]:
A = Vector([[2, 3]])
B = Vector([[1, 2], [3, 4]])

In [60]:
C = (A.tensor_product(B)).dagger()

In [61]:
D = A.dagger().tensor_product(B.dagger())

In [62]:
assert C == D